In [2]:
!python3 -m pip install beautifulsoup4 selenium pandas pyarrow Pillow requests cairosvg

import io, requests, pandas as pd, cairosvg # Useful libraries 
from selenium import webdriver
from selenium.webdriver import ChromeOptions # Finds Chrome browser to scrap from
from bs4 import BeautifulSoup # Scraps Images/Tables
from pathlib import Path # General housekeeping
from PIL import Image # Imports image onto VSCode
from urllib.parse import urljoin # Useful to link cutoff URL pathways 

# Stops selenium from opening GUI 
options = ChromeOptions()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)

# Makes URL into text readable by Python
url = "https://www.tpgtelecom.com.au"
driver.get(url)
content = driver.page_source
soup = BeautifulSoup(content, "html.parser")
driver.quit()

# Inspected and pinpointed the location of scraped image
def gets_url(classes, location, source):
    results = []
    for a in soup.find_all(attrs={"class": classes}):
        name = a.find(location)
        if name not in results:
            results.append(name.get(source))
    return results

# Extract image into repository
if __name__ == "__main__":
    returned_results = gets_url("img-fit-size", "img", "src")
    for b in returned_results:
        full_url = urljoin(url, b) # Image URL may not be linked to base URL
        response = requests.get(full_url) # Searches for images in URL section
        image_content = response.content
        content_type = response.headers.get("Content-Type", "")
        if content_type == "image/svg+xml": # Convert SVG to PNG if needed
            png_bytes = cairosvg.svg2png(bytestring=image_content)
            image = Image.open(io.BytesIO(png_bytes)).convert("RGB") 
        else: # For non-SVGs
            image = Image.open(io.BytesIO(image_content)).convert("RGB") 
        # Saves image into particular folder
        file_path = Path("/workspaces/FINM3422-Group-11/Scraped Images/images", "TPGcoverage" + ".png") 
        image.save(file_path, "PNG", quality=80)
